### Dataset Setup

In [20]:
# Basic imports
import numpy as np
import xarray as xr
import xdatasets as xd
from scipy.stats import genextreme, genpareto, gumbel_r

import xhydro as xh
from xhydro.extreme_value_analysis.parameterestimation import *

In [21]:
ds = (
    xd.Query(
        **{
            "datasets": {
                "deh": {
                    "id": ["020*"],
                    "regulated": ["Natural"],
                    "variables": ["streamflow"],
                }
            },
            "time": {"start": "1970-01-01", "minimum_duration": (15 * 365, "d")},
        }
    )
    .data.squeeze()
    .load()
)

# This dataset lacks some of the aforementioned attributes, so we need to add them.
ds["id"].attrs["cf_role"] = "timeseries_id"
ds["streamflow"].attrs = {
    "long_name": "Streamflow",
    "units": "m3 s-1",
    "standard_name": "water_volume_transport_in_river_channel",
    "cell_methods": "time: mean",
}

# Some examples
timeargs = {
    "spring": {"date_bounds": ["02-11", "06-19"]},
    "summer": {"doy_bounds": [152, 243]},
    "fall": {"month": [9, 10, 11]},
    "winter": {
        "season": ["DJF"],
        "freq": "YS-DEC",
    },  # To correctly wrap around the year, we need to specify the resampling frequency.
    "august": {"month": [8]},
    "annual": {},
}
# Here, we hide years with more than 15% of missing data.
ds_4fa = xh.indicators.get_yearly_op(
    ds, op="max", timeargs=timeargs, missing="pct", missing_options={"tolerance": 0.15}
)

# Create a mask beforehand
import random

nyears = np.unique(ds.time.dt.year).size
dom_start = xr.DataArray(
    np.random.randint(1, 30, size=(nyears,)).astype("str"),
    dims=("year"),
    coords={"year": np.unique(ds.time.dt.year)},
)
dom_end = xr.DataArray(
    np.random.randint(1, 30, size=(nyears,)).astype("str"),
    dims=("year"),
    coords={"year": np.unique(ds.time.dt.year)},
)

mask = xr.zeros_like(ds["streamflow"])
for y in dom_start.year.values:
    # Random mask of dates per year, between April and June.
    mask.loc[
        {
            "time": slice(
                str(y) + "-04-" + str(dom_start.sel(year=y).item()),
                str(y) + "-06-" + str(dom_end.sel(year=y).item()),
            )
        }
    ] = 1
# The name of the indexer will be used to identify the variable created here
timeargs_custom = {"custom": {}}

# We use where() to mask the data that we want to ignore
masked = ds.where(mask == 1)
# Since we masked almost all of the year, our tolerance for missing data should be changed accordingly
missing = "at_least_n"
missing_options = {"n": 45}

# We use xr.merge() to combine the results with the previous dataset.
ds_4fa = xr.merge(
    [
        ds_4fa,
        xh.indicators.get_yearly_op(
            masked,
            op="max",
            timeargs=timeargs_custom,
            missing=missing,
            missing_options=missing_options,
        ),
    ]
)
# Get a daily volume from a daily streamflow
ds["volume"] = xh.indicators.compute_volume(ds["streamflow"], out_units="hm3")

# We'll take slightly different indexers
timeargs_vol = {"spring": {"date_bounds": ["04-30", "06-15"]}, "annual": {}}

# The operation that we want here is the sum, not the max.
ds_4fa = xr.merge(
    [
        ds_4fa,
        xh.indicators.get_yearly_op(
            ds,
            op="sum",
            input_var="volume",
            timeargs=timeargs_vol,
            missing="pct",
            missing_options={"tolerance": 0.15},
            interpolate_na=True,
        ),
    ]
)

ds_4fa = ds_4fa.assign_coords(temperature=("temperature", np.linspace(25, 35, 56)))
print(ds_4fa["temperature"].values.tolist())
print(type(ds_4fa["temperature"].values.tolist()))

[25.0, 25.181818181818183, 25.363636363636363, 25.545454545454547, 25.727272727272727, 25.90909090909091, 26.09090909090909, 26.272727272727273, 26.454545454545453, 26.636363636363637, 26.81818181818182, 27.0, 27.18181818181818, 27.363636363636363, 27.545454545454547, 27.727272727272727, 27.90909090909091, 28.09090909090909, 28.272727272727273, 28.454545454545453, 28.636363636363637, 28.81818181818182, 29.0, 29.18181818181818, 29.363636363636363, 29.545454545454547, 29.727272727272727, 29.90909090909091, 30.09090909090909, 30.272727272727273, 30.454545454545453, 30.636363636363637, 30.81818181818182, 31.0, 31.18181818181818, 31.363636363636363, 31.545454545454547, 31.727272727272727, 31.90909090909091, 32.09090909090909, 32.27272727272727, 32.45454545454545, 32.63636363636364, 32.81818181818182, 33.0, 33.18181818181818, 33.36363636363636, 33.54545454545455, 33.72727272727273, 33.90909090909091, 34.09090909090909, 34.27272727272727, 34.45454545454545, 34.63636363636364, 34.8181818181818

# extreme_value_analysis.parameterestimation.fit()

In [8]:
params = fit(ds_4fa, dist=genextreme, method="ml")
params

<xarray.Dataset> Size: 4kB
Dimensions:                (id: 5, dparams: 9)
Coordinates: (12/15)
    drainage_area          (id) float32 20B 1.09e+03 647.0 59.8 626.0 1.2e+03
    end_date               (id) datetime64[ns] 40B 2006-10-13 ... 1996-08-13
  * id                     (id) object 40B '020302' '020404' ... '020802'
    latitude               (id) float32 20B 48.77 48.81 48.98 48.98 49.2
    longitude              (id) float32 20B -64.52 -64.92 -64.43 -64.7 -65.29
    name                   (id) object 40B 'Saint' 'York' ... 'Madeleine'
    ...                     ...
    spatial_agg            <U9 36B 'watershed'
    start_date             (id) datetime64[ns] 40B 1989-08-12 ... 1970-01-01
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
  * dparams                (dparams) <U11 396B 'shape' ... 'scale_upper'
Data variables:
    streamflow_max_spring  (id, dparams) float64 360B nan nan ... 51.45 98.12
    streamflow_max_summer  (id, dparams) float64 360B 0.4454 -0.2299 ... 88.42
    streamflow_max_fall    (id, dparams) float64 360B 0.8062 -0.2569 ... 27.84
    streamflow_max_august  (id, dparams) float64 360B 0.9015 0.01491 ... 18.39
    streamflow_max_annual  (id, dparams) float64 360B nan nan ... 51.38 100.1
    streamflow_max_winter  (id, dparams) float64 360B nan nan ... 6.664 17.17
    streamflow_max_custom  (id, dparams) float64 360B -0.1497 -0.6879 ... 116.7
    volume_sum_spring      (id, dparams) float64 360B -0.6526 -1.086 ... 174.3
    volume_sum_annual      (id, dparams) float64 360B -0.2645 -0.5601 ... 218.5
Attributes:
    cat:frequency:         yr
    cat:processing_level:  indicators
    cat:id:                
    long_name:             genextreme parameters
    description:           Parameters of the genextreme distribution
    method:                ML
    estimator:             Maximum likelihood
    scipy_dist:            genextreme
    units:                 
    history:               [2024-07-24 15:47:09] fit: Estimate distribution p...